In [1]:
import pandas as pd
import requests

from datetime import datetime
import time

from scrapy import Selector

In [48]:
url = 'https://en.wikipedia.org/wiki/Nasdaq-100'

html = requests.get(url).text
sel = Selector(text = html)

nasdaq100_df = pd.DataFrame()

for n in range(2,104):
    ticker = sel.xpath('//table[@id = "constituents"]/tbody/tr[{}]/td[2]'.format(n)).extract()[0].strip('</td>')
    company_name = sel.xpath('//table[@id = "constituents"]/tbody/tr[{}]/td[1]/a/@title'.format(n)).get()
    industry = sel.xpath('//table[@id = "constituents"]/tbody/tr[{}]/td[3]'.format(n)).get().strip('</td>')
    sub_industry = sel.xpath('//table[@id = "constituents"]/tbody/tr[{}]/td[3]'.format(n)).get().strip('</td>')
    new_list = [ticker, company_name, industry, sub_industry]
    pd_series_row = pd.Series(new_list, index = ['ticker', 'company_name', 'industry', 'sub_industry'])
    nasdaq100_df = nasdaq100_df.append(pd_series_row, ignore_index=True)
    print(ticker, end = ' ')

nasdaq100_df.head(4)

ATVI ADBE AMD ALGN GOOGL GOOG AMZN AEP AMGN ADI ANSS AAPL AMAT ASML TEAM ADSK ADP BIDU BIIB BKNG AVGO CDNS CDW CERN CHTR CHKP CTAS CSCO CTSH CMCSA CPRT COST CRWD CSX DXCM DOCU DLTR EBAY EA EXC FAST FISV FOXA FOX GILD HON IDXX ILMN INCY INTC INTU ISRG JD KDP KLAC KHC LRCX LULU MAR MRVL MTCH MELI FB MCHP MU MSFT MRNA MDLZ MNST NTES NFLX NVDA NXPI ORLY OKTA PCAR PAYX PYPL PTON PEP PDD QCOM REGN ROST SGEN SIRI SWKS SPLK SBUX SNPS TMUS TSLA TXN TCOM VRSN VRSK VRTX WBA WDAY XEL XLNX ZM 

,company_name,industry,sub_industry,ticker
0,Activision Blizzard,Communication Services,Communication Services,ATVI
1,Adobe Inc.,Information Technology,Information Technology,ADBE
2,Advanced Micro Devices,Information Technology,Information Technology,AMD
3,Align Technology,Health Care,Health Care,ALGN


In [49]:
def unix_to_date(unix_timestamp):
    ts = int(unix_timestamp)
    return datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')

In [50]:
with open('/Users/ilya/Desktop/keys/yahoo_finance.txt','r') as file:
    key = file.readlines()

KEY = key[0]

In [51]:
def get_weekly_history_df(symbol):
    get_history_url = "https://yh-finance.p.rapidapi.com/stock/v3/get-historical-data"

    querystring = {"symbol":symbol,"region":"US"}

    headers = {
    'x-rapidapi-host': "yh-finance.p.rapidapi.com",
    'x-rapidapi-key': KEY
    }

    response = requests.request("GET", get_history_url, headers=headers, params=querystring)

    json_response = response.json()

    weekly_history_df = pd.DataFrame(json_response['prices'])[['date','close']]
    weekly_history_df.loc[:,'date'] = weekly_history_df['date'].apply(lambda x: unix_to_date(x))

    weekly_history_df.loc[:,'week'] = [pd.Timestamp(date).week for date in weekly_history_df['date']]
    weekly_history_df.loc[:,'year'] = [pd.Timestamp(date).year for date in weekly_history_df['date']]

    weekly_history_df.loc[:,'close'] = weekly_history_df['close'].round(1)

    return weekly_history_df[['date','year','week','close']]

In [160]:
cash_flow_url = "https://yh-finance.p.rapidapi.com/stock/v2/get-balance-sheet"

querystring = {"symbol":"ATVI","region":"US"}

headers = {
    'x-rapidapi-host': "yh-finance.p.rapidapi.com",
    'x-rapidapi-key': "f31c8be57dmsh32b4005c3178b54p1f4272jsnb55bf64ce33e"
    }

response = requests.request("GET", cash_flow_url, headers=headers, params=querystring)

json_response = response.json()

In [161]:
##these are quarterly earnings specifically

for quarter in json_response['earnings']['earningsChart']['quarterly']:
    print(quarter)

{'date': '4Q2020', 'actual': {'raw': 1.21, 'fmt': '1.21'}, 'estimate': {'raw': 1.18, 'fmt': '1.18'}}
{'date': '1Q2021', 'actual': {'raw': 0.84, 'fmt': '0.84'}, 'estimate': {'raw': 0.7, 'fmt': '0.70'}}
{'date': '2Q2021', 'actual': {'raw': 0.91, 'fmt': '0.91'}, 'estimate': {'raw': 0.76, 'fmt': '0.76'}}
{'date': '3Q2021', 'actual': {'raw': 0.72, 'fmt': '0.72'}, 'estimate': {'raw': 0.7, 'fmt': '0.70'}}


In [157]:
for item in json_response['balanceSheetHistoryQuarterly']['balanceSheetStatements']:
    print(item['endDate'])
    print(item['totalCurrentLiabilities'])
    print(item['totalCurrentAssets'])

{'raw': 1632960000, 'fmt': '2021-09-30'}
{'raw': 2016000000, 'fmt': '2.02B', 'longFmt': '2,016,000,000'}
{'raw': 11211000000, 'fmt': '11.21B', 'longFmt': '11,211,000,000'}
{'raw': 1625011200, 'fmt': '2021-06-30'}
{'raw': 2253000000, 'fmt': '2.25B', 'longFmt': '2,253,000,000'}
{'raw': 10887000000, 'fmt': '10.89B', 'longFmt': '10,887,000,000'}
{'raw': 1617148800, 'fmt': '2021-03-31'}
{'raw': 3181000000, 'fmt': '3.18B', 'longFmt': '3,181,000,000'}
{'raw': 10922000000, 'fmt': '10.92B', 'longFmt': '10,922,000,000'}
{'raw': 1609372800, 'fmt': '2020-12-31'}
{'raw': 3100000000, 'fmt': '3.1B', 'longFmt': '3,100,000,000'}
{'raw': 10565000000, 'fmt': '10.56B', 'longFmt': '10,565,000,000'}


In [159]:
#this is the same as above but per year

for item in json_response['balanceSheetHistory']['balanceSheetStatements']:
    print(item['endDate'])
    print(item['totalCurrentLiabilities'])
    print(item['totalCurrentAssets'])

{'raw': 1609372800, 'fmt': '2020-12-31'}
{'raw': 3100000000, 'fmt': '3.1B', 'longFmt': '3,100,000,000'}
{'raw': 10565000000, 'fmt': '10.56B', 'longFmt': '10,565,000,000'}
{'raw': 1577750400, 'fmt': '2019-12-31'}
{'raw': 2915000000, 'fmt': '2.92B', 'longFmt': '2,915,000,000'}
{'raw': 7292000000, 'fmt': '7.29B', 'longFmt': '7,292,000,000'}
{'raw': 1546214400, 'fmt': '2018-12-31'}
{'raw': 2642000000, 'fmt': '2.64B', 'longFmt': '2,642,000,000'}
{'raw': 6106000000, 'fmt': '6.11B', 'longFmt': '6,106,000,000'}
{'raw': 1514678400, 'fmt': '2017-12-31'}
{'raw': 3663000000, 'fmt': '3.66B', 'longFmt': '3,663,000,000'}
{'raw': 6520000000, 'fmt': '6.52B', 'longFmt': '6,520,000,000'}


In [25]:
def list_perfomance(symbol):
    weekly_history_df = get_weekly_history_df(symbol)

    performance_list = [symbol,\
                        round((weekly_history_df.iloc[0]['close'] - weekly_history_df.iloc[-1]['close'])/weekly_history_df.loc[0]['close'],3),\
                        round(weekly_history_df['close'].std(),2),\
                        round(weekly_history_df['close'].var(),2),
                        round(weekly_history_df['close'].mean(),2),
                        round(weekly_history_df['close'].std()/weekly_history_df['close'].mean(),2),
                        round(weekly_history_df.iloc[0]['close'],1)
                        ]

    return performance_list

In [26]:
counter = 0

attributes = ['ticker','yoy_growth','std','variance','mean_price','volatility','last_close_price']
comparison_df = pd.DataFrame(columns = attributes)

for ticker_name in nasdaq100_list[64:67]:
    series_row = pd.Series(list_perfomance(ticker_name), index = attributes)
    comparison_df = comparison_df.append(series_row, ignore_index=True)
    print(ticker_name, end = ' ')
    time.sleep(1)

In [27]:
comparison_df.sort_values(by = 'yoy_growth', ascending=False).head(20)

,ticker,yoy_growth,std,variance,mean_price,volatility,last_close_price
